In [1]:
import gensim.downloader

glove_vectors = gensim.downloader.load('glove-twitter-25')
glove_vectors.most_similar('capitalism')

[('socialism', 0.9675610661506653),
 ('tyranny', 0.9390194416046143),
 ('inequality', 0.9126962423324585),
 ('oppression', 0.9065089225769043),
 ('greed', 0.9040176868438721),
 ('fascism', 0.8969089388847351),
 ('capitalist', 0.8934274911880493),
 ('democracy', 0.8926239013671875),
 ('morality', 0.8892533183097839),
 ('poverty', 0.877461314201355)]

In [2]:
from xml.dom import minidom
import psycopg2
conn = psycopg2.connect(
    host="localhost",
    database="datamining",
    user="msbean",
    password="cactusgreen")

conn.autocommit = True
cursor = conn.cursor() 

nones = 0
somes = 0
for v in range(50):
    dom = minidom.parse("../videos/" + str(v) + "_captions.txt")
    for caption in dom.getElementsByTagName("s"):
        for child_node in caption.childNodes:
            if child_node.nodeType == child_node.TEXT_NODE:
                stripped = ''.join([i for i in child_node.data if i.isalpha()]).lower()
                if stripped in glove_vectors:
                    embedding = glove_vectors[stripped]
                    
                    sql = "INSERT INTO caption_embeddings (video_path, word, embedding) VALUES ('"
                    sql += "../videos/" + str(v) + ".mp4', '"
                    sql += stripped
                    sql += "', "
                    sql += "'["
                    for datum in embedding:
                        sql += str(datum) + ","
                    sql = sql[:-1]
                    sql += "]')"

                    cursor.execute(sql) 
                    somes += 1
                else:
                    nones += 1
print(somes / (somes + nones))

conn.commit() 
conn.close() 

0.9795010714430114


In [13]:
#SELECT id, embedding FROM embeddings ORDER BY embedding <-> '[2

conn = psycopg2.connect(
    host="localhost",
    database="datamining",
    user="msbean",
    password="cactusgreen")

conn.autocommit = True
cursor = conn.cursor() 

tree_embedding = glove_vectors["tree"]
sql = "SELECT word, video_path FROM caption_embeddings ORDER BY embedding <-> "
sql += "'["
for datum in tree_embedding:
    sql += str(datum) + ","
sql = sql[:-1]
sql += "]' LIMIT 10"
cursor.execute(sql) 
records = cursor.fetchall()

print("Words similar to 'tree' fetched from database (and videos they appeared in)")
print(records)

Words similar to 'tree' fetched from database (and videos they appeared in)
[('tree', '../videos/37.mp4'), ('flowers', '../videos/41.mp4'), ('dark', '../videos/7.mp4'), ('palm', '../videos/17.mp4'), ('sun', '../videos/35.mp4'), ('stone', '../videos/19.mp4'), ('mountain', '../videos/15.mp4'), ('inside', '../videos/16.mp4'), ('light', '../videos/14.mp4'), ('wood', '../videos/10.mp4')]
